[ResNet of roboflow](https://colab.research.google.com/github/QIN2DIM/hcaptcha-challenger/blob/main/automation/roboflow_resnet.ipynb)

In [ ]:
!nvidia-smi

In [ ]:
!pip install loguru onnx fire
!git clone -b workflow https://github.com/captcha-challenger/hcaptcha-model-factory.git
!mv -f hcaptcha-model-factory/src .
!rm -rf hcaptcha-model-factory/

Set `label_name` and `GITHUB_TOKEN`

In [ ]:
import os

os.environ["GITHUB_TOKEN"] = ""

# After Annotating, edit `focus_flags`
# - Copy from: `[PROJECT]/database2309/<diagnosed_label_name>`
# - Paste to: `[PROJECT]/data/<diagnosed_label_name>`
# - Output to: `[PROJECT]/model/<diagnosed_label_name>/<model_name[flag].onnx>`

# fmt:off
focus_flags = {
    # "<diagnosed_label_name>": "<model_name[flag]>"
    # "fox": "fox2310",
}
# fmt:on

In [ ]:
from pathlib import Path


this_dir = Path("/content")
project_dir = this_dir

model_dir = project_dir.joinpath("model")
binary_dir = project_dir.joinpath("database2309")
factory_data_dir = project_dir.joinpath("data")
scripts_dir = project_dir.joinpath("scripts")

binary_dir.mkdir(parents=True, exist_ok=True)

Training

In [ ]:
%cd {src_dir}

import os
import shutil

from apis.scaffold import Scaffold
from loguru import logger


def quick_train():
    if not focus_flags:
        logger.warning("Skip model training, miss focus-flags")
        return

    # Copy the classified data into the dataset
    for task_name in focus_flags:
        to_dir = factory_data_dir.joinpath(task_name)
        shutil.rmtree(to_dir, ignore_errors=True)
        to_dir.mkdir(parents=True, exist_ok=True)

        # Flush dataset
        src_dir = binary_dir.joinpath(task_name)
        for hook in ["yes", "bad"]:
            shutil.copytree(src_dir.joinpath(hook), to_dir.joinpath(hook))

        # train
        Scaffold.train(task=task_name)


quick_train()

Deployment model to GitHub

In [ ]:
import locale

locale.getpreferredencoding = lambda d=True: "UTF-8"

!pip install PyGithub

from github import Auth, Github, GithubException


def quick_development():
    if not os.getenv("GITHUB_TOKEN"):
        logger.warning("Skip model deployment, miss GITHUB TOKEN")
        return

    auth = Auth.Token(os.getenv("GITHUB_TOKEN", ""))
    repo = Github(auth=auth).get_repo("QIN2DIM/hcaptcha-challenger")
    modelhub_title = "ONNX ModelHub"

    for label, onnx_archive in focus_flags.items():
        model_path = model_dir.joinpath(label, f"{label}.onnx")
        pending_onnx_path = model_dir.joinpath(label, f"{onnx_archive}.onnx")
        shutil.copy(model_path, pending_onnx_path)
        for release in repo.get_releases():
            if release.title != modelhub_title:
                continue
            try:
                res = release.upload_asset(path=str(pending_onnx_path))
            except GithubException as err:
                if err.status == 422:
                    logger.error(
                        f"The model file already exists, please manually replace the file with the same name - url={repo.releases_url}",
                        url=repo.releases_url,
                    )
            except Exception as err:
                logger.error(err)
            else:
                logger.success(
                    f"Model file uploaded successfully - name={res.name} url={res.browser_download_url}"
                )


quick_development()